In [2]:
# where is it ?

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import TargetEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

In [3]:
train_df_full = pd.read_csv('train.csv')#'/kaggle/input/playground-series-s4e9/train.csv')
test_df_full = pd.read_csv('test.csv')#'/kaggle/input/playground-series-s4e9/test.csv')

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pandas as pd


# Define PyTorch model with more capacity
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # Increased to 128 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)  # Increased second layer to 64 neurons
        self.fc3 = nn.Linear(64, 1)  # Output layer remains the same
        self.dropout = nn.Dropout(0.3)  # Dropout to prevent overfitting

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)
    

# Function to preprocess data and handle NaNs
def preprocess_data(X, y=None, target_encoder=None, scaler=None, nan_columns=None):
    # Remove 'id' column if it exists
    if 'id' in X.columns:
        X = X.drop(columns=['id'])
    
    # If nan_columns is provided (during test), use it; otherwise detect from the data (during train)
    if nan_columns is None:
        # Training phase: Identify columns with NaNs and store them
        nan_columns = X.columns[X.isna().any()].tolist()
    
    # Handle missing values (NaNs) by adding is_nan columns and imputing
    for col in nan_columns:
        X[f'{col}_is_nan'] = X[col].isna().astype(int)  # Add 'is_nan' column
        # this be done automatically i think, and we want consistent if new one is found later 
        #X[col].fillna(X[col].mean(), inplace=True)  # Impute NaNs with the column mean (or other strategy)

    # Apply TargetEncoder and StandardScaler
    if target_encoder is not None:
        if y is not None:
            X = target_encoder.fit_transform(X, y)
        else:
            X = target_encoder.transform(X)
    if scaler is not None:
        if y is not None:
            X = scaler.fit_transform(X)
        else:
            X = scaler.transform(X)
    
    return X, nan_columns


# Function to evaluate PyTorch model
def evaluate_pytorch_model(model, X_test, y_test):
    # Convert test data to PyTorch tensors
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)#.squeeze()  # Use squeeze to flatten the prediction shape
        rmse = torch.sqrt(torch.mean((predictions - y_test_tensor) ** 2)).item()
    return rmse

# Function to train and evaluate Ridge regression
def train_and_evaluate_ridge(X_train, y_train, X_test, y_test):
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_train, y_train)
    predictions = ridge_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return rmse

# Function to train PyTorch model and print training/validation loss at each epoch
def train_pytorch_model(X_train, y_train, X_test, y_test, input_size, epochs=20, batch_size=64, learning_rate=0.001):
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

    # Create DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Initialize PyTorch model, loss, and optimizer
    model = SimpleNN(input_size=input_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model and track losses
    model.train()
    for epoch in range(epochs):
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * batch_X.size(0)  # Accumulate training loss

        # Calculate average training loss
        train_loss = train_loss / len(train_loader.dataset)

        # Validation loss (RMSE) after each epoch
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)#.squeeze()  # Use squeeze to flatten the prediction shap
            val_loss = torch.sqrt(torch.mean((predictions - y_test_tensor) ** 2)).item()  # RMSE

        print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {train_loss:.0f}, Validation RMSE: {val_loss:.0f}')

    return model

# Function for cross-validation loop (with printing training/validation errors)
def cross_validate(X, y, target_encoder, scaler, n_splits=5, n_repeats=1, epochs=20, batch_size=64):
    kfold = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=SEED)
    rmse_scores_nn = []
    rmse_scores_ridge = []

    for train_index, test_index in kfold.split(X):
        print('A fold')
        # Split the data
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Preprocess the data
        X_train_scaled, nan_columns = preprocess_data(X_train, y_train, target_encoder, scaler)
        X_test_scaled, _ = preprocess_data(X_test, target_encoder=target_encoder, scaler=scaler, nan_columns=nan_columns)

        # Train and evaluate PyTorch model, printing the training/validation loss
        model = train_pytorch_model(X_train_scaled, y_train, X_test_scaled, y_test, input_size=X_train_scaled.shape[1], epochs=epochs, batch_size=batch_size)
        rmse_nn = evaluate_pytorch_model(model, X_test_scaled, y_test)
        rmse_scores_nn.append(rmse_nn)

        # Train and evaluate Ridge regression
        rmse_ridge = train_and_evaluate_ridge(X_train_scaled, y_train, X_test_scaled, y_test)
        rmse_scores_ridge.append(rmse_ridge)

    print(f'PyTorch Neural Network CV RMSE score: {np.mean(rmse_scores_nn):.0f} ± {np.std(rmse_scores_nn):.0f}')
    print(f'Ridge Regression CV RMSE score: {np.mean(rmse_scores_ridge):.0f} ± {np.std(rmse_scores_ridge):.0f}')



# Prepare data
SEED = 5
DEV_FRACTION = 1
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']


# Cross-validation setup
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)

# Initialize TargetEncoder and StandardScaler
target_encoder = TargetEncoder(random_state=SEED, target_type='continuous').set_output(transform='pandas')
scaler = StandardScaler()


# Run cross-validation
cross_validate(X, y, target_encoder, scaler, epochs = 20)


(188533, 13)
A fold


KeyboardInterrupt: 

# producing the output

In [8]:
# Main function to train on full data and make a submission
def train_full_model_and_submit(train_df, test_df, target_encoder, scaler, epochs=20, batch_size=64):
    # Separate features and target
    X_train = train_df.drop(columns='price')
    y_train = train_df['price']
    X_test = test_df
    
    # Preprocess the data
    X_train_scaled, nan_columns = preprocess_data(X_train, y_train, target_encoder, scaler)

    # Train the PyTorch model on the full training data
    model = train_pytorch_model(X_train_scaled, y_train, input_size=X_train_scaled.shape[1], epochs=epochs, batch_size=batch_size)

    # Prepare the test data and make predictions
    X_test_scaled, _ = preprocess_data(X_test, target_encoder=target_encoder, scaler=scaler, nan_columns=nan_columns)

    # Make predictions
    model.eval()
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    with torch.no_grad():
        predictions = model(X_test_tensor).numpy().flatten()

    # Create submission DataFrame
    submission = pd.DataFrame({
        'id': test_df['id'],
        'price': predictions
    })

    # Save submission to CSV
    submission.to_csv('submission10_Blended.csv', index=False)


# Train the full model and create a submission
train_full_model_and_submit(train_df_full, test_df_full, target_encoder, scaler)

(188533, 14)
(125690, 14)


In [9]:
train_df_full.isna().sum(axis=0)

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [10]:
test_df_full.isna().sum(axis=0)

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        3383
engine              0
transmission        0
ext_col             0
int_col             0
accident         1632
clean_title     14239
dtype: int64

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from skorch import NeuralNetRegressor
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
import pandas as pd

# Define PyTorch model with more capacity
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # Increased to 128 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)  # Increased second layer to 64 neurons
        self.fc3 = nn.Linear(64, 1)  # Output layer remains the same
        self.dropout = nn.Dropout(0.3)  # Dropout to prevent overfitting

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x.squeeze(-1)  # Remove the last dimension

# Function to train CatBoost model
def train_catboost_model(X_train, y_train):
    catboost_model = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.1, loss_function='RMSE', silent=True)
    catboost_model.fit(X_train, y_train)
    return catboost_model

# Function to create the ensemble
def create_ensemble(X_train, y_train, X_test, y_test, input_size, epochs=20, batch_size=64):
    # Convert data to float32 for PyTorch compatibility
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    y_train = y_train.astype(np.float32)  # Ensure target values are also float32
    y_test = y_test.astype(np.float32)

    # Wrap PyTorch model with skorch
    pytorch_model = NeuralNetRegressor(
        module=SimpleNN,
        module__input_size=input_size,
        max_epochs=epochs,
        lr=0.001,
        batch_size=batch_size,
        optimizer=optim.Adam,
        criterion=nn.MSELoss,
        device='cpu'  # Use 'cuda' if you have a GPU available
    )

    # Train Ridge model
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_train, y_train)

    # Train CatBoost model
    catboost_model = train_catboost_model(X_train, y_train)

    # Create the ensemble using VotingRegressor
    ensemble = VotingRegressor([
        ('nn', pytorch_model),
        ('ridge', ridge_model),
        ('catboost', catboost_model)
    ])
    
    # Train the ensemble
    ensemble.fit(X_train, y_train)


    # Predict on the test set
    predictions = np.column_stack([
        est.predict(X_test).squeeze() if hasattr(est, 'predict') else est.predict(X_test)
        for name, est in ensemble.named_estimators_.items()
    ])
    rmse = np.sqrt(mean_squared_error(y_test, predictions.mean(axis=1)))

    # Predict on the test set
    #predictions = ensemble.predict(X_test)
    
    # Calculate RMSE for the ensemble
    #rmse = np.sqrt(mean_squared_error(y_test, predictions))
    
    return rmse

# Function to train PyTorch model (using skorch wrapper)
def train_pytorch_model(X_train, y_train, input_size, epochs=20, batch_size=64):
    X_train = X_train.astype(np.float32)  # Convert data to float32
    y_train = y_train.astype(np.float32)  # Convert target to float32
    pytorch_model = NeuralNetRegressor(
        module=SimpleNN,
        module__input_size=input_size,
        max_epochs=epochs,
        lr=0.001,
        batch_size=batch_size,
        optimizer=optim.Adam,
        criterion=nn.MSELoss,
        device='cpu'  # Use 'cuda' if you have a GPU available
    )
    pytorch_model.fit(X_train, y_train)
    return pytorch_model

# Cross-validation setup
def cross_validate_ensemble(X, y, target_encoder, scaler, n_splits=5, n_repeats=1, epochs=20, batch_size=64):
    kfold = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=SEED)
    rmse_scores_ensemble = []

    for train_index, test_index in kfold.split(X):
        print('a fold')
        # Split the data
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Preprocess the data
        X_train_scaled, nan_columns = preprocess_data(X_train, y_train, target_encoder, scaler)
        X_test_scaled, _ = preprocess_data(X_test, target_encoder=target_encoder, scaler=scaler, nan_columns=nan_columns)

        # Train and evaluate the ensemble
        rmse_ensemble = create_ensemble(X_train_scaled, y_train, X_test_scaled, y_test, input_size=X_train_scaled.shape[1], epochs=epochs, batch_size=batch_size)
        rmse_scores_ensemble.append(rmse_ensemble)

    print(f'Ensemble CV RMSE score: {np.mean(rmse_scores_ensemble):.5f} ± {np.std(rmse_scores_ensemble):.5f}')

# Prepare data
SEED = 5
DEV_FRACTION = .1
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']

# Cross-validation setup
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)

# Initialize TargetEncoder and StandardScaler
target_encoder = TargetEncoder(random_state=SEED, target_type='continuous').set_output(transform='pandas')
scaler = StandardScaler()

# Run cross-validation
cross_validate_ensemble(X, y, target_encoder, scaler)


(18853, 13)
a fold
  epoch       train_loss       valid_loss     dur
-------  ---------------  ---------------  ------
      1  7937223691.4792  9673196129.2410  0.1585
      2  7756225891.9705  9285550054.5442  0.1543
      3  7137827504.8235  8461583321.3073  0.1530
      4  6360444649.1902  7761897463.0056  0.1549
      5  5910015679.2255  7440568706.2486  0.1530
      6  5702013150.7455  7294467748.4441  0.1525
      7  5582267252.6375  7206747142.9579  0.1525
      8  5536552714.1212  7150708266.2565  0.1533
      9  5482051016.8481  7109609947.8528  0.1515
     10  5446911179.4845  7079921701.3351  0.1535
     11  5411764544.3395  7056994434.3334  0.1558
     12  5384878958.3092  7041607920.9811  0.1530
     13  5360365606.3151  7028960992.5197  0.1543
     14  5364115970.6841  7020504671.2894  0.1521
     15  5357649046.7567  7014822946.6198  0.1562
     16  5352735773.7058  7008527906.7895  0.1553
     17  5310024244.1071  7002930339.7653  0.1598
     18  5326788596.1973  69995

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pandas as pd

# Define PyTorch model with hyperparameters for tuning
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden1_size=128, hidden2_size=64, dropout_prob=0.3):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, 1)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x.squeeze(-1)  # Remove the last dimension

# Function to train PyTorch model with tunable hyperparameters
def train_pytorch_model(X_train, y_train, input_size, epochs=20, batch_size=64, learning_rate=0.001, hidden1_size=128, hidden2_size=64, dropout_prob=0.3):
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

    # Create DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Initialize PyTorch model, loss, and optimizer
    model = SimpleNN(input_size=input_size, hidden1_size=hidden1_size, hidden2_size=hidden2_size, dropout_prob=dropout_prob)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    model.train()
    for epoch in range(epochs):
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    return model

# Function to evaluate PyTorch model
def evaluate_pytorch_model(model, X_test, y_test):
    # Convert test data to PyTorch tensors
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        rmse = torch.sqrt(torch.mean((predictions - y_test_tensor) ** 2)).item()
    return rmse

# Hyperparameter optimization loop
def hyperparameter_search(X, y, target_encoder, scaler, param_grid):
    best_params = None
    best_score = float('inf')
    for params in param_grid:
        print(f"Testing with params: {params}")
        rmse_scores_nn = []

        # Cross-validation
        kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)
        for train_index, test_index in kfold.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Preprocess the data
            X_train_scaled, nan_columns = preprocess_data(X_train, y_train, target_encoder, scaler)
            X_test_scaled, _ = preprocess_data(X_test, target_encoder=target_encoder, scaler=scaler, nan_columns=nan_columns)

            # Train and evaluate PyTorch model with current hyperparameters
            model = train_pytorch_model(X_train_scaled, y_train, input_size=X_train_scaled.shape[1], **params)
            rmse_nn = evaluate_pytorch_model(model, X_test_scaled, y_test)
            rmse_scores_nn.append(rmse_nn)

        mean_rmse = np.mean(rmse_scores_nn)
        print(f"Mean RMSE for params {params}: {mean_rmse:.5f}")
        
        # Update best score and parameters
        if mean_rmse < best_score:
            best_score = mean_rmse
            best_params = params

    print(f"Best RMSE: {best_score:.5f} with params: {best_params}")
    return best_params, best_score

# Prepare data
SEED = 4
DEV_FRACTION = .5
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']

# Initialize TargetEncoder and StandardScaler
target_encoder = TargetEncoder(random_state=SEED, target_type='continuous').set_output(transform='pandas')
scaler = StandardScaler()

# Define parameter grid for hyperparameter search
param_grid = [
    {
        'epochs': [20, 30],               # Number of epochs
        'batch_size': [64, 128],          # Batch size
        'learning_rate': [0.001, 0.0005], # Learning rate
        'hidden1_size': [128, 256],       # First hidden layer size
        'hidden2_size': [64, 128],        # Second hidden layer size
        'dropout_prob': [0.3, 0.5]        # Dropout probability
    }
]

# Run hyperparameter search
best_params, best_score = hyperparameter_search(X, y, target_encoder, scaler, param_grid)


Testing with params: {'epochs': [20, 30], 'batch_size': [64, 128], 'learning_rate': [0.001, 0.0005], 'hidden1_size': [128, 256], 'hidden2_size': [64, 128], 'dropout_prob': [0.3, 0.5]}


ValueError: batch_size should be a positive integer value, but got batch_size=[64, 128]

# above the hyper param needs to implement the lists into singles
# here, we are doing another submission for blended model 

In [6]:
# Main function to train the ensemble on full data and make a submission
def train_full_model_and_submit(train_df, test_df, target_encoder, scaler, epochs=20, batch_size=64):
    # Separate features and target
    X_train = train_df.drop(columns='price')
    y_train = train_df['price']
    X_test = test_df

    # Preprocess the data
    X_train_scaled, nan_columns = preprocess_data(X_train, y_train, target_encoder, scaler)
    X_test_scaled, _ = preprocess_data(X_test, target_encoder=target_encoder, scaler=scaler, nan_columns=nan_columns)

    # Train the ensemble model (PyTorch, Ridge, and CatBoost)
    X_train_scaled = X_train_scaled.astype(np.float32)  # Convert to float32 for PyTorch
    y_train = y_train.astype(np.float32)  # Ensure y_train is also float32
    ensemble = VotingRegressor([
        ('nn', NeuralNetRegressor(
            module=SimpleNN,
            module__input_size=X_train_scaled.shape[1],
            max_epochs=epochs,
            lr=0.001,
            batch_size=batch_size,
            optimizer=optim.Adam,
            criterion=nn.MSELoss,
            device='cpu'  # Use 'cuda' if you have a GPU available
        )),
        ('ridge', Ridge(alpha=1.0)),
        ('catboost', CatBoostRegressor(
            iterations=500, depth=6, learning_rate=0.1, loss_function='RMSE', silent=True
        ))
    ])
    
    # Train the ensemble on the full training data
    ensemble.fit(X_train_scaled, y_train)

    # Predict on the test set
    X_test_scaled = X_test_scaled.astype(np.float32)  # Ensure X_test is also float32
    predictions = ensemble.predict(X_test_scaled)

    # Create the submission DataFrame
    submission = pd.DataFrame({
        'id': test_df['id'],
        'price': predictions
    })

    # Save the submission to CSV
    submission.to_csv('submission10_Blended_Ensemble.csv', index=False)

    print("Submission file created as 'submission10_Blended_Ensemble.csv'")

# Train the full model and create a submission
train_full_model_and_submit(train_df_full, test_df_full, target_encoder, scaler)


  epoch       train_loss       valid_loss     dur
-------  ---------------  ---------------  ------
      1  6289850170.4341  5186077151.2727  1.8258
      2  5476816694.6605  5110998660.9188  1.8278
      3  5436950200.7715  5089125104.3781  1.7541
      4  5424680796.8710  5079573866.3934  1.7917
      5  5411299301.0219  5075067791.4929  1.7567
      6  5409756922.9280  5072396486.5329  1.7667
      7  5405006535.7576  5071179457.2645  1.8669
      8  5401908786.4410  5070165521.7096  1.7954
      9  5404455016.4331  5069738445.3051  1.8207
     10  5403479674.0573  5069055797.8553  1.7612
     11  5404216668.7929  5068479951.0669  1.6904
     12  5398717384.3237  5068327056.0666  1.6903
     13  5402430333.7523  5067968267.2463  1.8123
     14  5406663231.8027  5067631694.7716  1.7337
     15  5397039569.6025  5067364400.3628  1.7398
     16  5399562416.8514  5067130723.2410  1.6901
     17  5403133463.1667  5066912036.3935  1.7162
     18  5400754948.1419  5066673057.5114  1.8244
